<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install kaggle

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [6]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip


Archive:  train_features.csv.zip
replace train_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
replace train_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
replace test_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif, SelectKBest

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

from mlxtend.plotting import plot_decision_regions
import category_encoders as ce


In [2]:

df_train = pd.read_csv('train_features.csv')
df_test = pd.read_csv('test_features.csv')
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 40), (14358, 40), (59400, 2))

In [78]:
# majority baseline
# balanced data - logistic class_weight = 
#            (None, accuracy = 0.55),
#            (balanced,accuracy=0.39-0.4)


df_label['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [41]:
# All data
df_sample = df_train.sample(2000)
y_sample = df_label[df_label.id.isin(df_sample.id)]

df_sample = pd.get_dummies(df_sample)

X_all = df_sample.drop('id',axis=1)
y = y_sample.drop('id',axis=1)
X_all.shape, y.shape


((2000, 6268), (2000, 1))

In [3]:
# Numeric data only
df_sample = df_train.sample(2000)
y_sample = df_label[df_label.id.isin(df_sample.id)]

X_num = df_sample._get_numeric_data().drop(['id'],axis=1)
y = y_sample.drop('id',axis=1)
X_num.shape, y.shape

((2000, 9), (2000, 1))

In [0]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
        X_num, y, test_size=0.25, random_state=42, shuffle=True)

In [106]:
# Baseline metric - numeric only - test_score=0.55

scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None), X_train_val, y_train_val, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

,estimator,fit_time,score_time,test_score,train_score
0,"LogisticRegression(C=1.0, class_weight=None, d...",1.179935,0.015035,0.553835,0.550456
1,"LogisticRegression(C=1.0, class_weight=None, d...",1.359063,0.023087,0.550168,0.552492
2,"LogisticRegression(C=1.0, class_weight=None, d...",1.452206,0.022775,0.550677,0.551968


In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

In [32]:
!pip install category_encoders

In [12]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="logreg")
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=LogisticRegression, step=1, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X_train_val, y_train_val)

print("Optimal number of features : %d" % rfecv.n_features_)

TypeError: ignored

In [10]:

from sklearn.feature_selection import RFECV

logr_u = LogisticRegression(solver='lbfgs')
# rank all features, i.e continue the elimination until the last one
rfecv = RFECV(logr_u)
rfecv.fit(X_train_val, y_train_val)

# Make negative so it works better for comparison to other scores after scaling
#rfe_u_score = pd.Series(data=(-1*rfe_u.ranking_), name='RFE_unbal', index=names)

rfecv.ranking_

array([9, 8, 2, 1, 5, 4, 3, 6, 7])

In [40]:
# Define an estimator and param_grid
pipe = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    #MinMaxScaler(), 
    SelectKBest(f_classif), 
    LogisticRegression(solver='lbfgs'))

param_grid = {
    'selectkbest__k': range(1, 11)
    #'selectkbest__k': range(1, len(X_train_val.columns)+1)
    #'selectkbest__k': [1,2,3,4],
    #'logisticregression__class_weight': [None,'balanced'],
    #'logisticregression__C': [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', 
                  verbose=1)

gs.fit(X_train_val, y_train_val)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: ignored

In [88]:
# Predict with X_test features
y_pred = gs.predict(X_test)

# Compare predictions to y_test labels
test_score = accuracy_score(y_test, y_pred)
print('Test Score:', test_score)

('Test Score:', 0.5603367003367004)
